In [4]:
import numpy as np
import oskar
from oskar import Sky
import astropy





In [5]:
oskar_skymodel_format= r'/home/yash-op-45/Documents/simulation-ska/test_files/altaz to radec to OSKAR SkyModel format_copy.csv'
data = np.genfromtxt(oskar_skymodel_format, delimiter=',', skip_header=1)
print(data.shape)
skymodel = Sky()
sky=Sky.load(data)


(19, 13)


TypeError: argument 1 must be str, not numpy.ndarray

In [6]:
sky = oskar.Sky.from_array(data)
print(sky.num_sources)

RuntimeError: Too many source parameters.

In [7]:
# Specifying only RA, Dec and Stokes I (other columns default to 0).

data = np.array([[20.0, -30.0, 1],
                        [20.0, -30.5, 3],
                        [20.5, -30.5, 3]])

sky = oskar.Sky.from_array(data)

print(sky.num_sources)

3


In [8]:
sky = oskar.Sky.generate_random_power_law(100000, 1e-3, 100e-3, -2)

print(sky.num_sources)

100000


In [9]:
ra0 = 0
dec0 = 80
sky.filter_by_radius(5, 15, ra0, dec0)
print(sky.num_sources)

1429


In [2]:
from astropy.io import fits

hdulist = fits.open(r'/home/yash-op-45/Documents/simulation-ska/test_files/vogtstar_awt.fits')

cols = hdulist[1].data[0].array

data = np.column_stack(
        (cols['RAJ2000'], cols['DEJ2000'], cols['peak_flux_wide']))

sky = oskar.Sky.from_array(data)

print(sky.num_sources)
307455

AttributeError: 'numpy.float32' object has no attribute 'array'

OSKAR Example Script- "hello world"

In [13]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy
import oskar

In [14]:
params = {
    "simulator": {
        "use_gpus": True
    },
    "observation" : {
        "num_channels": 3,
        "start_frequency_hz": 100e6,
        "frequency_inc_hz": 20e6,
        "phase_centre_ra_deg": 20,
        "phase_centre_dec_deg": -30,
        "num_time_steps": 24,
        "start_time_utc": "01-01-2000 12:00:00.000",
        "length": "12:00:00.000"
    },
    "telescope": {
        "input_directory": "telescope.tm"
    },
    "interferometer": {
        "oskar_vis_filename": "example.vis",
        "ms_filename": "",
        "channel_bandwidth_hz": 1e6,
        "time_average_sec": 10
    }
}
settings = oskar.SettingsTree("/home/yash-op-45/Documents/simulation-ska/test_files/oskar_sim_interferometer.ini")
settings.from_dict(params)

RuntimeError: oskar_app_settings_tree() failed.

In [ ]:
precision = "single"
if precision == "single":
    settings["simulator/double_precision"] = False


In [ ]:
sky_data = numpy.array([
        [20.0, -30.0, 1, 0, 0, 0, 100.0e6, -0.7, 0.0, 0,   0,   0],
        [20.0, -30.5, 3, 2, 2, 0, 100.0e6, -0.7, 0.0, 600, 50,  45],
        [20.5, -30.5, 3, 0, 0, 2, 100.0e6, -0.7, 0.0, 700, 10, -10]])
sky = oskar.Sky.from_array(sky_data, precision)

In [ ]:
sim = oskar.Interferometer(settings=settings)
sim.set_sky_model(sky)
sim.run()

In [ ]:
imager = oskar.Imager(precision)
imager.set(fov_deg=4, image_size=512)
imager.set(input_file="/home/yash-op-45/Documents/simulation-ska/test_files/example.vis", output_root="example")
output = imager.run(return_images=1)
image = output["images"][0]